In [21]:
# Adjust atomic term energies (SO-CI matrix diagonals) to fit exptl energy levels
#    to obtain semiempirical term energies
# This version has better assignments, to match theoretical and exptl levels
# Also allows multiple terms with same basis symbol
# KKI 7/12/2023
import re, sys, glob, subprocess
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy, collections
#sys.path.insert(0, '../karlib')
import chem_subs as chem
import molpro_subs as mpr

pd.set_option('display.max_rows', None)

In [22]:
# Excel spreadsheet of experimental levels from https://physics.nist.gov/PhysRefData/ASD/levels_form.html
#   Download as CSV; paste into a column in Excel; use Data -> Text to Columns -> Delimited -> Comma
#   Rename that worksheet with a name like "Fe" or "Fe+"
# Note that experimental levels might not be listed by increasing energy
xl_expt = 'exptl_levels.xlsx'
xl = pd.ExcelFile(xl_expt, engine='openpyxl')

### Select atom and parity of interest

In [23]:
atom = 'Ni'  # a name like "Fe" or "Fe+"
parity = 'even'  #  choose 'even' or 'odd' or 'both'

In [24]:
Ecol = 'Level (cm-1)'  # the exptl energy column
# display formatting
fmt = {'Eshift': '{:.1f}', Ecol: '{:.3f}', 'Pct': '{:.3f}', 'degen': '{:.0f}'}
for col in ['J', 'Ecalc', 'E_dif', 'Erel', 'Eshift', 'err', 'Eterm', 'cm-1', 'fitted']:
    fmt[col] =  fmt['Eshift']
for col in ['dif', 'Theory', 'ecm', 'SOC', 'RMSE']:
    fmt[col] = '{:.2f}'

In [25]:
require_standard_symbols = True  # make False to allow ASD-type alternative "term" symbols

In [26]:
if atom not in xl.sheet_names:
    print(f'No experimental data sheet for {atom}!')
else:
    dfexpt = pd.read_excel(xl, atom, engine='openpyxl')
    # Delete any ionization limit and above
    ilim = dfexpt[dfexpt.Term == 'Limit'].index.min()
    # delete the "Limit" row
    dfexpt = dfexpt[dfexpt.Term != 'Limit']
    # delete everything past the Limit
    n1 = len(dfexpt)
    dfexpt = dfexpt.truncate(after=ilim)
    n2 = len(dfexpt)
    if n2 < n1:
        print(f'Discarded {n1-n2} metastable states')
    dfeven = dfexpt[~dfexpt.Term.str.contains('\*$')].copy()
    dfodd = dfexpt[dfexpt.Term.str.contains('\*$')].copy()
    print(f'{len(dfexpt)} experimental levels ({len(dfeven)} even and {len(dfodd)} odd) for {atom} read from "{xl_expt}"')
    # Select by parity
    if parity == 'even':
        # discard odd levels ('Term' field ends with '*')
        dfexpt = dfeven.copy()
    elif parity == 'odd':
        dfexpt = dfodd.copy()
    print(f'{len(dfexpt)} levels are of parity "{parity}"')
    # add degeneracy = 2J+1
    dfexpt['degen'] = 2 * dfexpt.J + 1
# Check for unrecognized term labels
bad_labels = []
if require_standard_symbols:
    for trm in dfexpt.Term:
        try:
            chem.possible_J_from_term(trm)
        except:
            bad_labels.append(trm)
else:
    # allow alternative "term" symbols like "2[5/2]"
        try:
            chem.possible_J_from_ASD_label(trm)
        except:
            bad_labels.append(trm)
if bad_labels:
    print('Some term labels are not recognizable!')
    print('The following levels will be deleted:')
    display(dfexpt[dfexpt.Term.isin(bad_labels)].style.format(fmt))
    dfexpt = dfexpt[~dfexpt.Term.isin(bad_labels)]
else:
    print('All term labels are recognizable')
# Create unique term labels, "uTerm"
dfexpt = chem.unique_labels_exptl_terms(dfexpt, verbose=True, always=True)

Discarded 2 metastable states
285 experimental levels (128 even and 157 odd) for Ni read from "exptl_levels.xlsx"
128 levels are of parity "even"
Some term labels are not recognizable!
The following levels will be deleted:


,Configuration,Term,J,Prefix,Level (cm-1),Suffix,Uncertainty (cm-1),Lande,Leading percentages,Reference,degen
59,3d9.(2D<5/2>).5s,2[5/2],3.0,nan,42605.945,nan,nan,1.340000,100,nan,7
60,3d9.(2D<5/2>).5s,2[5/2],2.0,nan,42790.010,nan,nan,1.085000,99,nan,5
81,3d9.(2D<3/2>).5s,2[3/2],1.0,nan,44112.173,nan,nan,1.090000,100,nan,3
82,3d9.(2D<3/2>).5s,2[3/2],2.0,nan,44262.599,nan,nan,nan,99,nan,5
104,3d9.(2D<5/2>).4d,2[1/2],1.0,nan,48953.316,nan,nan,1.920000,99,nan,3
105,3d9.(2D<5/2>).4d,2[1/2],0.0,nan,49610.345,nan,nan,nan,74 : 22 3d9.(2D<5/2>).4d 2[1/2],nan,1
107,3d9.(2D<5/2>).4d,2[9/2],5.0,nan,49158.480,nan,nan,1.200000,100,nan,11
108,3d9.(2D<5/2>).4d,2[9/2],4.0,nan,49174.770,nan,nan,1.050000,99,nan,9
109,3d9.(2D<5/2>).4d,2[3/2],2.0,nan,49159.030,nan,nan,1.430000,98,nan,5
110,3d9.(2D<5/2>).4d,2[3/2],1.0,nan,49171.151,nan,nan,1.000000,99,nan,3


### Specify Molpro SO-CI output file

In [27]:
fsoc = 'fe_15Q21T_ctzdk_x2c.pro'  # use termcut=20000
fsoc = 'fe_ci_15Q7T_c5zdk_x2c.pro'  # use termcut=18000
fsoc = 'fe_15Q7T_ctzdk_x2c.pro'  # use termcut=18000
fsoc = r'../UMemphis/NI15T21S-cc-TZ-DK.out'

print(f'Reading MOLPRO file "{fsoc}"')
compAtom = mpr.stoichiometry(fsoc)
charge = mpr.total_charge(fsoc)
print(f'The atom is {compAtom} with charge {charge}')
# check for consistency with the experimental data that were read
if charge > 0: 
    compAtom += '+'
elif charge < 0:
    compAtom += '-'
if abs(charge) > 1:
    compAtom += f'{abs(charge)}'

if compAtom != atom:
    print(f'*** exptl atom = {atom} is different')
PG = mpr.read_compgroup(fsoc)
print(f'The computational point group is {PG}')

Reading MOLPRO file "../UMemphis/NI15T21S-cc-TZ-DK.out"
The atom is Ni with charge 0.0
The computational point group is Ci


In [28]:
SOCI = mpr.fullmatSOCI(fsoc, hybrid=True, sortval=False)
vals_original = SOCI.vals.copy()

Computational group = Ci
CASSCF states:
    21 Singlet
    15 Triplet
Replacing MRCI+Q energies by HLSDIAG values


In [29]:
dfterm = SOCI.average_terms(be_close=['Energy', 'Edav', 'Eref', 'dipZ', 'C0'], always=True)
# drop the dipZ column
dfterm.drop(columns=['dipZ'], inplace=True)
print('Averaged terms:')
dfso = SOCI.assign_atomic_J(quiet=True)  # create SOCI.dfso
Egl = SOCI.dfso.E.min()  # energy of ground level
dfterm['Erel'] = (dfterm.Edav - Egl) * chem.AU2CM
display(dfterm.style.format(fmt))

Averaged terms:


,Term,Edav,idx,ecm,Erel
0,(1)3F,-1519.415799,[0 1 4 5 2 6 3],0.00,1004.4
1,(1)3D,-1519.412704,[ 9 8 10 11 7],679.20,1683.6
2,(1)1D,-1519.402251,[15 16 18 17 19],2973.50,3977.9
3,(1)1S,-1519.394967,[34],4572.20,5576.6
4,(2)1D,-1519.347992,[21 22 20 24 23],14882.00,15886.4
5,(1)3P,-1519.346195,[12 14 13],15276.30,16280.7
6,(1)1G,-1519.311238,[25 30 32 27 28 29 26 31 33],22948.50,23952.9
7,(2)1S,-1519.179310,[35],51903.50,52907.9


In [30]:
# Discard experimental levels whose leading terms are not among those computed
noncompos = set(dfexpt.uTerm) - set(dfterm.Term)
if len(noncompos):
    print('Some exptl terms were not computed:', noncompos)
    print('Deleting the following levels, led by those terms:')
    dfn = dfexpt[dfexpt.uTerm.isin(noncompos)]
    display(dfn.style.format(fmt))
    dfexpt = dfexpt[~dfexpt.uTerm.isin(noncompos)]

Some exptl terms were not computed: {'(1)5F', '(2)5F', '(1)1F', '(3)3F', '(2)3F'}
Deleting the following levels, led by those terms:


,Configuration,Term,J,Prefix,Level (cm-1),Suffix,Uncertainty (cm-1),Lande,Leading percentages,Reference,degen,uTerm
91,3d8.4s.(4F).5s,5F,5.0,nan,48466.490,nan,nan,1.400000,100,nan,11,(1)5F
92,3d8.4s.(4F).5s,5F,4.0,nan,49085.982,nan,nan,1.330000,80 : 11 3d8.4s.(4F).5s 3F,nan,9,(1)5F
93,3d8.4s.(4F).5s,5F,3.0,nan,49777.569,nan,nan,1.230000,88 : 8 3d8.4s.(4F).5s 3F,nan,7,(1)5F
94,3d8.4s.(4F).5s,5F,2.0,nan,50346.427,nan,nan,0.950000,94,nan,5,(1)5F
95,3d8.4s.(4F).5s,5F,1.0,nan,50744.552,nan,nan,0.200000,93,nan,3,(1)5F
123,3d8.4s.(4F).5s,3F,4.0,nan,50466.131,nan,nan,nan,85 : 12 3d8.4s.(4F).5s 5F,nan,9,(2)3F
124,3d8.4s.(4F).5s,3F,3.0,nan,51306.038,nan,nan,nan,80 : 10 3d8.4s.(4F).5s 5F,nan,7,(2)3F
125,3d8.4s.(4F).5s,3F,2.0,nan,52040.523,nan,nan,nan,88 : 7 3d8.4s.(2F).5s 3F,nan,5,(2)3F
141,3d8.4s.(2F).5s,3F,4.0,nan,54237.099,nan,nan,1.270000,93,nan,9,(3)3F
142,3d8.4s.(2F).5s,3F,3.0,nan,55576.843,nan,nan,1.000000,69 : 26 3d8.4s.(2F).5s 1F,nan,7,(3)3F


In [39]:
def match_expt_theory_simple(dfexpt, dftheory):
    # match exptl and theoretical levels, based upon leading term
    # return a DataFrame containing both
    dfcomp = dfexpt[['Configuration', 'uTerm', 'J', Ecol, 'degen']].copy()
    dfcomp['Tcalc'] = ''  # term assignment in computation
    dfcomp['Ecalc'] = np.nan
    for i, row in dftheory.iterrows():
        term = row.Lead
        J = row.J
        for ix, rowx in dfexpt.iterrows():
            if (rowx.uTerm == term) and (rowx.J == J):
                if not np.isnan(dfcomp.at[ix, 'Ecalc']):
                    print('Already paired!', display(rowx.to_frame().T))
                    
                else:
                    dfcomp.at[ix, 'Ecalc'] = row.Erel
                    dfcomp.at[ix, 'Tcalc'] = term
    dfcomp['err'] = dfcomp.Ecalc - dfcomp[Ecol]
    return dfcomp

In [32]:
def levels_from_term_energies(term_order, term_energies):
    # Install the term energies along the SOCI.matrix diagonal and rediagonalize
    #   'term_order' is array of term symbols
    #   'term_energies' is array of corresponding energies (cm-1)
    # Return nothing
    #global SOCI
    #   'SOCI' is a fullmatSOCI() object and is modified
    term_dict = dict(zip(term_order, term_energies))
    newdiag = SOCI.matrix.diagonal().copy()
    for ibs in range(len(newdiag)):
        j = SOCI.sob_ici[ibs]
        term = SOCI.mrci[j].Term
        # install the new energy for the term
        newdiag[ibs] = term_dict[term]
    # update the matrix
    SOCI.fill_diagonal(newdiag)
    SOCI.diagonalize(store=True, vectors=True, sortval=False)  # update the SOCI() object
    SOCI.assign_atomic_J(quiet=True)
    return

In [33]:
def rmserr(dfcomp):
    # Given a matched-up comparison of expt and theory, return the
    #   rms error in level energies
    # Weighted with degeneracies
    rmse = np.dot(dfcomp.err**2, dfcomp.degen.astype(float))
    rmse /= dfcomp.degen.sum()
    rmse = np.sqrt(rmse)
    return rmse
def obj_fun(exc_terme):
    # Given only excited term energies (assuming ground=0)
    #   return the RMSE
    global term_order, dfexpt, SOCI
    terme = [0] + list(exc_terme)  # the fixed energy of the ground term is 0
    levels_from_term_energies(term_order, terme)
    dfcomp = match_expt_theory_simple(dfexpt, SOCI.dfso)
    rmse = rmserr(dfcomp)
    return rmse

In [34]:
# Create global 'term_order' 
term_order = dfterm.Term.values
term_energies = dfterm.ecm.values

In [35]:
print('Initial comparison with expt, before fitting')
dfcomp = match_expt_theory_simple(dfexpt, SOCI.dfso)
rmse0 = rmserr(dfcomp)
print(f'RMSE = {rmse0:.1f} cm-1')
display(dfcomp.style.format(fmt))

Initial comparison with expt, before fitting
RMSE = 1738.2 cm-1


,Configuration,uTerm,J,Level (cm-1),degen,Tcalc,Ecalc,err
0,3d8.(3F).4s2,(1)3F,4.0,0.000,9,(1)3F,0.0,0.0
1,3d8.(3F).4s2,(1)3F,3.0,1332.164,7,(1)3F,1333.4,1.3
2,3d8.(3F).4s2,(1)3F,2.0,2216.550,5,(1)3F,2240.3,23.7
3,3d9.(2D).4s,(1)3D,3.0,204.787,7,(1)3D,1073.2,868.4
4,3d9.(2D).4s,(1)3D,2.0,879.816,5,(1)3D,1744.7,864.8
5,3d9.(2D).4s,(1)3D,1.0,1713.087,3,(1)3D,2599.1,886.0
6,3d9.(2D).4s,(1)1D,2.0,3409.937,5,(1)1D,4226.3,816.4
7,3d8.(1D).4s2,(2)1D,2.0,13521.347,5,(2)1D,16888.8,3367.4
8,3d10,(1)1S,0.0,14728.840,1,(1)1S,5558.1,-9170.7
9,3d8.(3P).4s2,(1)3P,2.0,15609.844,5,(1)3P,14997.1,-612.8


In [36]:
def freport(xvec):
    # callback function to monitor minimization
    freport.counter += 1
    print(f'{freport.counter:5d}', end='')
    return
freport.counter = 0

In [37]:
# Minimize the RMSE
exc_terme = list(term_energies)[1:]  # only excited terms; assume ground term = 0 energy
result = scipy.optimize.minimize(obj_fun, exc_terme, method='Nelder-Mead', callback=freport)

    1    2    3    4    5    6    7    8    9   10   11   12   13   14   15   16   17   18   19   20   21   22   23   24   25   26   27   28   29   30   31   32   33   34   35   36   37   38   39   40   41   42   43   44   45   46   47   48   49   50   51   52   53   54   55   56   57   58   59   60   61   62   63   64   65   66   67   68   69   70   71

,Configuration,Term,J,Prefix,Level (cm-1),Suffix,Uncertainty (cm-1),Lande,Leading percentages,Reference,degen,uTerm
9,3d8.(3P).4s2,3P,2,NaN,15609.844,NaN,NaN,1.356,78 : 18 3d8.(1D).4s2 ...,NaN,5,(1)3P


Already paired! None
   72   73   74   75   76   77   78   79   80   81   82   83   84   85   86   87   88   89   90   91   92   93   94   95   96   97   98   99  100  101  102  103  104  105  106  107  108  109  110  111  112  113  114  115  116  117  118  119  120  121  122  123  124  125  126  127  128  129  130  131  132  133  134  135  136  137  138  139  140  141  142  143  144  145  146  147  148  149  150  151  152  153  154  155  156  157  158  159  160  161  162  163  164  165  166  167  168  169  170  171  172  173  174  175  176  177  178  179  180  181  182  183  184  185  186  187  188  189  190  191  192  193  194  195  196  197  198  199  200  201  202  203  204  205  206  207  208  209  210  211  212  213  214  215  216  217  218  219  220  221  222  223  224  225  226  227  228  229  230  231  232  233  234  235  236  237  238  239  240  241  242  243  244J values found are not J values expected!
Expected: [(0.0, 3), (1.0, 2), (2.0, 5), (3.0, 2), (4.0, 2)]
Found   : [

In [41]:
if result.success:
    print(f'Minimization complete in {result.nit} iterations with {result.nfev} evaluations')
    print(f'RMSE = {result.fun:.1f} after Nelder-Mead')
    rmse = result.fun
    print('Fitted levels')
    fit_terme = [0] + list(result.x)
    levels_from_term_energies(term_order, fit_terme)
    dfcomp = match_expt_theory_simple(dfexpt, SOCI.dfso)
    rmse = rmserr(dfcomp)
    display(dfcomp.style.format(fmt))
    print(f'RMSE = {rmse:.1f} cm-1 compared with initial RMSE = {rmse0:.1f}')
else:
    print('*** Failure ***')
    print(result)

Minimization complete in 886 iterations with 1350 evaluations
RMSE = 18.2 after Nelder-Mead
Fitted levels


,Configuration,uTerm,J,Level (cm-1),degen,Tcalc,Ecalc,err
0,3d8.(3F).4s2,(1)3F,4.0,0.000,9,(1)3F,0.0,0.0
1,3d8.(3F).4s2,(1)3F,3.0,1332.164,7,(1)3F,1334.8,2.7
2,3d8.(3F).4s2,(1)3F,2.0,2216.550,5,(1)3F,2223.7,7.1
3,3d9.(2D).4s,(1)3D,3.0,204.787,7,(1)3D,200.2,-4.6
4,3d9.(2D).4s,(1)3D,2.0,879.816,5,(1)3D,878.2,-1.6
5,3d9.(2D).4s,(1)3D,1.0,1713.087,3,(1)3D,1726.1,13.0
6,3d9.(2D).4s,(1)1D,2.0,3409.937,5,(1)1D,3410.0,0.1
7,3d8.(1D).4s2,(2)1D,2.0,13521.347,5,(2)1D,13525.0,3.6
8,3d10,(1)1S,0.0,14728.840,1,(1)1S,14711.8,-17.1
9,3d8.(3P).4s2,(1)3P,2.0,15609.844,5,(1)3P,15600.9,-8.9


RMSE = 18.2 cm-1 compared with initial RMSE = 1738.2


In [43]:
# Add to Term DataFrame
print('Fitted term energies; "Erel" is relative to the lowest level')
dfterm['fitted'] = np.nan
for term, terme in zip(term_order, fit_terme):
    dfterm.at[dfterm.Term == term, 'fitted'] = terme
Eshift = SOCI.dfso.Eshift.min()  # energy of ground level relative to lowest term
dfterm['Erel'] = (dfterm.fitted - Eshift) # relative to ground level
dfterm.style.format(fmt)

Fitted term energies; "Erel" is relative to the lowest level


,Term,Edav,idx,ecm,Erel,fitted
0,(1)3F,-1519.415799,[0 1 4 5 2 6 3],0.00,1005.9,0.0
1,(1)3D,-1519.412704,[ 9 8 10 11 7],679.20,810.6,-195.3
2,(1)1D,-1519.402251,[15 16 18 17 19],2973.50,3161.7,2155.8
3,(1)1S,-1519.394967,[34],4572.20,14876.1,13870.2
4,(2)1D,-1519.347992,[21 22 20 24 23],14882.00,14048.0,13042.1
5,(1)3P,-1519.346195,[12 14 13],15276.30,15347.7,14341.8
6,(1)1G,-1519.311238,[25 30 32 27 28 29 26 31 33],22948.50,22083.1,21077.2
7,(2)1S,-1519.179310,[35],51903.50,50200.1,49194.2


In [20]:
1/0

ZeroDivisionError: division by zero

In [ ]:
cruft below

In [ ]:
def align_expt_theory(warnThresh=800, showDF=True, silent=False):
    # Compare experimental levels with corresponding theoretical
    #   'warnThresh' will display larger discrepancies in red
    global enforce_term_assignment

    dfdiff = dfexpt.copy()
    dfdiff['Ecalc'] = np.nan
    # match computed levels to exptl
    dflevel = SOCI.dfso
    idx = list(dflevel.index)  # list of computed, assigned levels
    termwt = []  # for arrays of term weights (from theory)
    leading = []  # leading term in the calculation
    failmatch = False
    ifail = []  # list of exptl rows that cannot be matched
    for i, row in dfexpt.iterrows():
        #display(dfexpt.loc[i].to_frame().T)
        while idx:
            # there are theoretical levels that have not been matched to exptl
            for j in idx.copy():
                if float(row.J) != float(dflevel.at[j, 'J']):
                    # values of J don't match
                    continue
                if enforce_term_assignment:
                    # Also require that terms have same label
                    # ** This may be problematic for ambiguous term labeling **
                    if row.Tlbl != dflevel.loc[j, 'Lead']:
                        #print('<<unequal term:', row.Term, dflevel.loc[j, 'Leading'])
                        continue
                # J (and optionally Term) match
                #print('>>>matches:')
                #display(dflevel.loc[j].to_frame().T)
                dfdiff.at[i, 'Ecalc'] = dflevel.at[j, 'Erel']
                termwt.append(dflevel.at[j, 'termwt'])
                leading.append(dflevel.at[j, 'Lead'])
                idx.remove(j)
                break
            else:
                if not silent:
                    print('Failed to assign any theoretical level to this exptl:')
                    display(row.to_frame().T)
                    print('Rejected possibilities:')
                    display(dflevel.loc[idx].style.format(fmt))
                ifail.append(i)
                failmatch = True
            break
    if failmatch:
        if enforce_term_assignment:
            print('*** Try setting enforce_term_assignment = False ***')
        else:
            # just delete the non-matching rows
            if not silent:
                print('Stop tracking these exptl levels:')
                display(dfexpt.loc[ifail])
            dfdiff.drop(ifail, inplace=True)
    dfdiff['err'] = np.round(dfdiff.Ecalc - dfdiff[Ecol], 2)
    try:
        dfdiff['termwt'] = termwt
    except ValueError:
        print('*** Error ***  Try adjusting "termcut" in cell #4 ***')
        1/0
    # keep only some columns
    dfdiff = dfdiff[['Configuration', 'Term', Ecol, 'Tlbl', 'J',
                     'degen', 'Ecalc', 'err', 'termwt']]
    if not enforce_term_assignment:
        # report the theoretical term 
        dfdiff['Leading'] = leading
    if showDF:
        display(dfdiff.drop('termwt', axis=1).style.apply(lambda x: ["background: yellow" if abs(v) > warnThresh else "" for v in x], 
                          subset=pd.IndexSlice[['err']]).format(fmt))
        print(f'Errors > {warnThresh} cm-1 are highlighted')
    return dfdiff

In [ ]:
def compute_rmse2(dfexpt, degen=True, DFret=False, silent=False):
    # Given SO-CI level energies return their RMS error
    #   'dfexpt' is the DataFrame of exptl level energies
    #   'degen' is whether to include (2J+1) weights (i.e. sublevels individually)
    #   'DFret' is whether to return the DataFrame
    # Assign values of J using energy and term composition
    #global term_order, SOCI
    SOCI.assign_atomic_J(quiet=True)    
    dfdiff = align_expt_theory(showDF=False, silent=silent)
    dfdiff = dfdiff.sort_values('Level (cm-1)')
    if degen:
        rmse = np.dot(dfdiff.degen, dfdiff.err**2)
        rmse /= dfdiff.degen.sum()
    else:
        rmse = (dfdiff.err**2).mean()
    rmse = np.sqrt(rmse)
    if DFret:
        return rmse, dfdiff
    else:
        return rmse

In [ ]:
def rmse_fun(term_energies, silent=False):
    # Uses globals
    # Return RMSE given term energies
    # Do not allow the lowest term energy to change
    #global dfexpt, SOCI, term_order
    levels_from_term_energies(term_order, term_energies)
    rmse = compute_rmse2(dfexpt, silent=silent)
    return rmse
def obj_fun(exc_terme):
    # Given only excited term energies (assuming ground=0)
    #   return the RMSE
    #global dfexpt, SOCI, term_order
    terme = [0] + list(exc_terme)  # the fixed energy of the ground term is 0
    rmse = rmse_fun(terme, silent=True)
    return rmse

In [ ]:
# Require that expt and calc have same leading terms? Global variable
enforce_term_assignment = False
if enforce_term_assignment:
    print('--When matching exptl/theor levels, term labels will be required to match--')

In [ ]:
SOCI.assign_atomic_J(quiet=True)
olddfso = SOCI.dfso.copy()
#olddfso.style.format(fmt)

In [ ]:
dferr = align_expt_theory(showDF=True)

In [ ]:
rmse_original, dforig = compute_rmse2(dfexpt, DFret=True)  # as received from MOLPRO
oldmat = SOCI.matrix.copy()
oldvals = SOCI.vals.copy()
SOCraw = vals_original[0]
print(f'Original RMSE = {rmse_original:.2f} cm-1 with theoretical SOCraw = {SOCraw:.2f} cm-1')
#display(dforig.style.format(fmt))

In [ ]:
# Summary DF for different data treatments
cols = ['Case', 'RMSE', 'SOC'] + list(dfeq1.Tlbl.values)
xtvals = list(dfeq1.Erel.values)
if not enforce_term_assignment:
    # add any theoretical terms that are missing
    for t in term_order:
        if t not in cols:
            cols.append(t)
            xtvals.append(np.nan)
dfsummary = pd.DataFrame(columns=cols)
for t in (tlbls + list(dfterm.Term)):
    fmt[t] = '{:.1f}'
# situation without theoretical info
row = ['expt only', np.nan, SOC1, *xtvals]
dfsummary.loc[len(dfsummary)] = row

In [ ]:
# Use the averaged term energies--expect little change
rmse_avgd = rmse_fun(term_energies)
SOCth = SOCI.dfso.Eshift.min()
print(f'Using averaged input term energies, RMSE = {rmse_avgd:.2f} cm-1 and SOCth = {SOCth:.2f} cm-1')
d = dict(zip(dfterm.Term, dfterm.ecm))
row = ['Before fit', rmse_avgd, SOCth] + [d.get(t, np.nan) for t in dfsummary.columns[3:]]
row.extend([np.nan] * (len(dfsummary.columns) - len(row)))
#display(dfsummary)
#display(row)
dfsummary.loc[len(dfsummary)] = row

In [ ]:
dfsummary.style.format(fmt)

In [ ]:
# Compute Hessian 
dE = 20  # step size / cm-1
def grad_fun(xcent):
    # two-sided gradient
    ndim = len(xcent)
    grad = np.zeros(ndim)
    for i in range(ndim):
        x = xcent.copy()
        x[i] += dE
        y1 = obj_fun(x)
        x[i] -= 2 * dE
        y_1 = obj_fun(x)
        grad[i] = (y1 - y_1) / (2 * dE)
    return grad
def hess_fun(xcent):
    # two-sided hessian
    ndim = len(xcent)
    hess = np.zeros((ndim, ndim))
    for i in range(ndim):
        x = xcent.copy()
        x[i] += dE
        y1 = grad_fun(x)
        x[i] -= 2 * dE
        y_1 = grad_fun(x)
        hess[i,:] = (y1 - y_1) / (2 * dE)
    return hess

In [ ]:
grad = grad_fun(result.x)
print('Gradient =', np.round(grad, 4))
hess = hess_fun(result.x)
print('Hessian:')
np.set_printoptions(suppress=True)
print(np.round(hess, 5))
print('Diagonal:')
print(np.round(hess.diagonal(), 5))

In [ ]:
rmse = result.fun
print(f'After minimization, RMSE = {rmse:.2f} cm-1')
terme = [0] + list(result.x)
Eterm = dict(zip(term_order, terme))
print('\nExptl vs fitted level energies:')
dfdiff = align_expt_theory()
#display(dfcomp.style.format(fmt))
SOCfit = SOCI.vals[0]
print(f'The lowest level energy = SOCfit = {SOCfit:.2f} cm-1')

In [ ]:
d = dict(zip(term_order, terme))
row = ['After fit', rmse, SOCfit] + [d.get(t, np.nan) for t in dfsummary.columns[3:]]
dfsummary.loc[len(dfsummary)] = row
display(dfsummary.style.format(fmt).hide_index())
print(f'Input file: {fsoc}')